In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt# Root directory of the project
ROOT_DIR = "/media/aspa2/KINGSTON/SummerResearch/MMXXII"

'''# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library'''
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config

%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
MODEL_PATH = os.path.join(MODEL_DIR, "object20220122T1721/mask_rcnn_object_3000.h5")

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError("The file \"{}\" is not found.".format(MODEL_PATH))

/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorf

In [6]:
class InferenceConfig(Config):    
    # Give the configuration a recognizable name
    NAME = "object"

    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    NUM_CLASSES = 1 + 1  # Background + rockmelon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [7]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
# Load weights trained
model.load_weights(MODEL_PATH, by_name=True)

<class 'str'>







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


2022-01-25 14:28:45.301369: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-01-25 14:28:45.306262: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3503890000 Hz
2022-01-25 14:28:45.306405: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x10735610 executing computations on platform Host. Devices:
2022-01-25 14:28:45.306417: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-01-25 14:28:45.308368: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2022-01-25 14:28:45.392249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-25 14:28:45.393392: I tensorflow/compiler/xla/service/serv

Re-starting from epoch 3000


In [8]:
class_names = ['BG', 'rockmelon']

In [1]:
# import libraries
import os, sys
import sys
import random
import math
import numpy as np
import scipy.misc

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

import matplotlib
import matplotlib.pyplot as plt# define random colors

%matplotlib inline

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

#apply mask to image
def apply_mask(image, mask, color, alpha=0.5):
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
        mask == 1,
        image[:, :, n] * (1 - alpha) + alpha * c,
        image[:, :, n]
        )
    
    return image

#take the image and apply the mask, box, and Label
def display_instances(image, boxes, masks, ids, names, scores):
    n_instances = boxes.shape[0]
    colors = random_colors(n_instances)

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue
    
    y1, x1, y2, x2 = boxes[i]
    label = names[ids[i]]
    score = scores[i] if scores is not None else None
    caption = '{} {:.2f}'.format(label, score) if score else label
    mask = masks[:, :, i]
    
    image = apply_mask(image, mask, color)
    image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    image = cv2.putText(
        image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
    )
    
    return image

/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorf

In [2]:
# Mask R-CNN
ROOT_DIR = "/media/aspa2/KINGSTON/SummerResearch/MMXXII/"
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
VIDEO_DIR = "/media/aspa2/KINGSTON/SummerResearch/recording/"
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "inference")
MODEL_PATH = os.path.join(MODEL_DIR, "object20220122T1721/mask_rcnn_object_3000.h5")

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError("The file \"{}\" is not found.".format(MODEL_PATH))

class InferenceConfig(Config):
    NAME = "object"

    NUM_CLASSES = 1 + 1  # Background + rockmelon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    GPU_COUNT = 1
    batch_size = 20
    IMAGES_PER_GPU = batch_size
    
config = InferenceConfig()
config.display()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(MODEL_PATH, by_name=True)
class_names = ['BG', 'rockmelon']


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     20
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 20
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE       

2022-01-26 10:31:06.305248: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-01-26 10:31:06.309771: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3503890000 Hz
2022-01-26 10:31:06.310022: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x10986600 executing computations on platform Host. Devices:
2022-01-26 10:31:06.310035: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-01-26 10:31:06.311844: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2022-01-26 10:31:06.399270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-26 10:31:06.400062: I tensorflow/compiler/xla/service/serv

Re-starting from epoch 3000


In [3]:
import cv2

batch_size = 20
video = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'row1_2022-01-12-12-54-49_camera_color_image_raw.mp4'))

# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if int(major_ver) < 3 :
    fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
else:
    fps = video.get(cv2.CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
try:
    if not os.path.exists(VIDEO_SAVE_DIR):
        os.makedirs(VIDEO_SAVE_DIR)
except OSError:
    print('Error: Creating directory of data')

frames = []
frame_count = 0

while True:
    ret, frame = video.read() 
    if not ret:
        break
    
    # Save each frame of the video to a list
    frame_count += 1
    frames.append(frame)
    print('frame_count :{0}'.format(frame_count))
    
    if len(frames) == batch_size:
        results = model.detect(frames, verbose=0)
        print('Predicted')
        for i, item in enumerate(zip(frames, results)):
            frame = item[0]
            r = item[1]
            try:
                frame = display_instances(
                    frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
                )
            except:
                pass
            name = '{0}.jpg'.format(frame_count + i - batch_size + 1)
            name = os.path.join(VIDEO_SAVE_DIR, name)
            cv2.imwrite(name, frame)
            print('writing to file:{0}'.format(name))
            # Clear the frames array to start the next batch
        frames = []

video.release()

Frames per second using video.get(cv2.CAP_PROP_FPS) : 25.0
frame_count :1
frame_count :2
frame_count :3
frame_count :4
frame_count :5
frame_count :6
frame_count :7
frame_count :8
frame_count :9
frame_count :10
frame_count :11
frame_count :12
frame_count :13
frame_count :14
frame_count :15
frame_count :16
frame_count :17
frame_count :18
frame_count :19
frame_count :20


2022-01-26 10:31:21.473265: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
2022-01-26 10:31:21.797089: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2022-01-26 10:31:22.122159: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudnn.so.7


Predicted
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/1.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/2.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/3.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/4.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/5.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/6.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/7.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/8.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/9.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/10.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/11.jpg
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference/12.jpg
writing to file:/media/aspa2/KINGSTON/S

KeyboardInterrupt: 

In [13]:
import cv2

batch_size = 20
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "test_inference")
video = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'row1_2022-01-12-12-54-49_camera_color_image_raw.mp4'))

# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if int(major_ver) < 3 :
    fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
else:
    fps = video.get(cv2.CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
try:
    if not os.path.exists(VIDEO_SAVE_DIR):
        os.makedirs(VIDEO_SAVE_DIR)
except OSError:
    print('Error: Creating directory of data')

frames = []
frame_count = 0

while True:
    ret, frame = video.read() 
    if not ret:
        break
    
    # Save each frame of the video to a list
    frame_count += 1
    frames.append(frame)
    print('frame_count :{0}'.format(frame_count))
    
    '''if len(frames) == batch_size:
        results = model.detect(frames, verbose=0)
        print('Predicted')
        for i, item in enumerate(zip(frames, results)):
            # frame = item[0]
            # r = results[0]
            frame = item[0]
            r = results[1]
            try:
                frame = display_instances(
                    frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
                )
            except:
                pass
            name = '{0}.jpg'.format(frame_count + i - batch_size + 1)
            name = os.path.join(VIDEO_SAVE_DIR, name)
            cv2.imwrite(name, frame)
            print('writing to file:{0}'.format(name))
            # Clear the frames array to start the next batch
            frames = []'''
    
    for i, item in enumerate(zip(frames, results)):
        filename = "rgb_image%s.jpg" %(i+1)
        print(filename)
        print(frames)

        # Run object detection
        results1 = model.detect(frames[i], verbose=1)

        # Display results
        # ax = get_ax(1)
        r1 = results1[0]
        img = visualize.display_instances(image, r1['rois'], r1['masks'], r1['class_ids'],
        dataset.class_names, r1['scores'], ax=ax, title=filename, save_image=True, save_dir="3000", save_filename=filename)
        img_path = os.path.join(VIDEO_SAVE_DIR, filename)
        cv2.imwrite(img_path, img)

video.release()

Frames per second using video.get(cv2.CAP_PROP_FPS) : 25.0
frame_count :1
rgb_image1.jpg
[array([[[166, 159, 165],
        [198, 192, 198],
        [204, 199, 202],
        ...,
        [108, 120, 118],
        [109, 122, 117],
        [109, 122, 117]],

       [[152, 145, 151],
        [212, 206, 212],
        [207, 201, 204],
        ...,
        [108, 120, 118],
        [109, 122, 117],
        [109, 122, 117]],

       [[133, 127, 132],
        [198, 192, 198],
        [204, 199, 202],
        ...,
        [106, 118, 116],
        [108, 120, 116],
        [108, 120, 116]],

       ...,

       [[133, 136, 126],
        [134, 137, 128],
        [134, 137, 128],
        ...,
        [154, 137, 140],
        [152, 137, 140],
        [151, 136, 139]],

       [[135, 138, 129],
        [135, 138, 129],
        [135, 138, 129],
        ...,
        [152, 137, 140],
        [152, 140, 142],
        [153, 141, 143]],

       [[135, 138, 129],
        [135, 138, 129],
        [135, 138, 129

AssertionError: len(images) must be equal to BATCH_SIZE

In [ ]:
#Create a video from a list of segmented images.
import glob
import os

def make_video(outvid, images=None, fps=30, size=None, is_color=True, format="FMP4"):
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
                vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
        img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

# Path Configuration
ROOT_DIR = os.getcwd()
VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "savedimgs")
images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))

# Sort the images by integer index
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

outvid = os.path.join(VIDEO_DIR, "video_inference.mp4")
make_video(outvid, images, fps=30)

In [ ]:
# https://toarches.medium.com/image-video-and-real-time-webcam-object-detection-and-instance-segmentation-with-mask-r-cnn-37a4675dcb49

In [1]:
# import libraries
import os, sys
import sys
import random
import math
import numpy as np
import scipy.misc

import mrcnn.model as modellib
from mrcnn import utils, visualize
from mrcnn.config import Config

import matplotlib
import matplotlib.pyplot as plt

/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aspa2/rockmelon/MMXXII/lib/python3.7/site-packages/tensorf

In [2]:
def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

colors = random

In [6]:
#video detection
import cv2
import numpy as np

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors
 
def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image
 
def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    colors = []
    
    n_instances = boxes.shape[0]
    # colors = random_colors(n_instances)
    # colors = [tuple(37, 150, 190) for _ in range(N)]
    for i in range(n_instances):
        colors.append(tuple((91, 193, 218)))
    # colors += n_instances * tuple(37, 150, 190)
    print(colors)
 
    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]
 
    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue
 
        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]
 
        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.6, color, 2
        )
 
    return image
 

if __name__ == '__main__':
    import os
    import sys
    
    from mrcnn import utils 
    from mrcnn import model as modellib
    from mrcnn.config import Config
    
    # We use a K80 GPU with 24GB memory, which can fit 3 images.
    batch_size = 2
 
    ROOT_DIR = "/media/aspa2/KINGSTON/SummerResearch/MMXXII/"
    MODEL_DIR = os.path.join(ROOT_DIR, "logs")
    VIDEO_DIR = "/media/aspa2/KINGSTON/SummerResearch/recording/"
    VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "inference_sc")
    MODEL_PATH = os.path.join(MODEL_DIR, "object20220122T1721/mask_rcnn_object_3000.h5")
    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError("The file \"{}\" is not found.".format(MODEL_PATH))
 
    class InferenceConfig(Config):
        NAME = 'object'
        NUM_CLASSES = 2
        STEPS_PER_EPOCH = 10
        DETECTION_MIN_CONFIDENCE = 0.9
        GPU_COUNT = 1
        IMAGES_PER_GPU = 2
        TRAIN_ROIS_PER_IMAGE = 512
 
    config = InferenceConfig()
    config.display()
 
    model = modellib.MaskRCNN(
        mode="inference", model_dir=MODEL_DIR, config=config
    )
    model.load_weights(MODEL_PATH, by_name=True)
    class_names = ['BG', 'rockmelon']
 
    capture = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'row1_2022-01-12-12-54-49_camera_color_image_raw.mp4'))
    try:
        if not os.path.exists(VIDEO_SAVE_DIR):
            os.makedirs(VIDEO_SAVE_DIR)
    except OSError:
        print ('Error: Creating directory of data')
    frames = []
    frame_count = 0
    # these 2 lines can be removed if you dont have a 1080p camera.
    # capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    # capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
 
    while True:
        ret, frame = capture.read()
        # Bail out when the video file ends
        if not ret:
            break
        
        # Save each frame of the video to a list
        frame_count += 1
        frames.append(frame)
        print('frame_count :{0}'.format(frame_count))
        if len(frames) == batch_size:
            results = model.detect(frames, verbose=0)
            print('Predicted')
            for i, item in enumerate(zip(frames, results)):
                frame = item[0]
                r = item[1]
                try:
                    frame = display_instances(
                        frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
                    )
                except:
                    pass
                name = '{0}.jpg'.format(frame_count + i - batch_size)
                name = os.path.join(VIDEO_SAVE_DIR, name)
                imghsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32") 
                
                (h, s, v) = cv2.split(imghsv) 
                s = s*1.5
                s = np.clip(s,0,255) 
                imghsv = cv2.merge([h,s,v]) 

                imgrgb = cv2.cvtColor(imghsv.astype("uint8"), cv2.COLOR_HSV2BGR)
                
                cv2.imwrite(name, imgrgb)
                print('writing to file:{0}'.format(name))
            # Clear the frames array to start the next batch
            frames = []
 
    capture.release()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/34.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/35.jpg
frame_count :37
frame_count :38
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/36.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/37.jpg
frame_count :39
frame_count :40
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/Sum

Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/72.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/73.jpg
frame_count :75
frame_count :76
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/74.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/75.jpg
frame_count :77
frame_count :78
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/reco

Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/112.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/113.jpg
frame_count :115
frame_count :116
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/114.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/115.jpg
frame_count :117
frame_count :118
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/116.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/Summer

Predicted
[(91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/164.jpg
[(91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/165.jpg
frame_count :167
frame_count :168
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/166.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/167.jpg
frame_count :169
frame_count :170
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/168.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/169.jpg
frame_count :171
frame_count :172
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 2

...

Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9460.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9461.jpg
frame_count :9463
frame_count :9464
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9462.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9463.jpg
frame_count :9465
frame_count :9466
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_

Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9498.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9499.jpg
frame_count :9501
frame_count :9502
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9500.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9501.jpg
frame_count :9503
frame_count :9504
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9502.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91

Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9544.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9545.jpg
frame_count :9547
frame_count :9548
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9546.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9547.jpg
frame_count :9549
frame_count :9550
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/as

Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9586.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9587.jpg
frame_count :9589
frame_count :9590
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9588.jpg
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9589.jpg
frame_count :9591
frame_count :9592
Predicted
[(91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218), (91, 193, 218)]
writing to file:/media/as

Predicted
[(91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9634.jpg
[(91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9635.jpg
frame_count :9637
frame_count :9638
Predicted
[(91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9636.jpg
[(91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9637.jpg
frame_count :9639
frame_count :9640
Predicted
[(91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9638.jpg
[(91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9639.jpg
frame_count :9641
frame_count :9642
Predicted
[(91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_sc/9640.jpg
[(91, 193, 218), (91, 193, 218)]
writing to file:/media/aspa2/KINGSTON/SummerResearch/recording/inference_

In [7]:
print(os.getcwd())

/home/aspa2/rockmelon/MMXXII


In [8]:
#Create a video from a list of segmented images.
import glob
import os

def make_video(outvid, images=None, fps=30, size=None, is_color=True, format="FMP4"):
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
                vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

# Path Configuration
images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))

# Sort the images by integer index
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

outvid = os.path.join(VIDEO_DIR, "video_inference_sc.mp4")
make_video(outvid, images, fps=30)

OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


<VideoWriter 0x7fb6ab98ceb0>